In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [25]:
# logistic regression
# y = sigmoid(WX + B)
# softmax regression
# y = softmax(WX +B)
# multi class classification
# cost(w) = -(ylogH(X) + (1-y)log(1-H(X))) -> -(y1log(p1) + y2log(p2))
# cost function
# if restart, get same result. random seed set
torch.manual_seed(1)
z = torch.FloatTensor([1,2,3])
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
print(hypothesis.sum())
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)
print(hypothesis.sum())
# randint max 5, size = (3,)
y = torch.randint(5, (3,)).long()
print(y)
# one-hot encoding
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1),1)
print(y.unsqueeze(1))
print(y_one_hot)
# cost
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)
# softmax function
# 1. F.softmax() + torch.log() = F.log_softmax()
# low level
# torch.log(F.softmax(z, dim=1))
# high level
# F.log_softmax(z, dim=1)
# 2. F.log_softmax() + F.nll_loss() = F.cross_entropy()
# low level
# (y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()
(y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()
# high level
# F.nll_loss(F.log_softmax(z, dim=1), y)
F.cross_entropy(z, y)

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)
tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)
tensor(3.0000, grad_fn=<SumBackward0>)
tensor([0, 2, 1])
tensor([[0],
        [2],
        [1]])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])
tensor(1.4689, grad_fn=<MeanBackward0>)


tensor(1.4689, grad_fn=<NllLossBackward0>)

In [ ]:
# softmax regression
# train data
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
# one-hot encoding
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

In [8]:
# low level way
# train data
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
# one-hot encoding
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
# W, b initial
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer
optimizer = optim.SGD([W, b], lr=0.1)
# training
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # hypothesis
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 
    # cost function
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    # cost -> hypothesis update
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # print log
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
print(W)
print(b)

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962
tensor([[-3.6761, -0.1014,  3.7776],
        [-0.6601, -0.4641,  1.1242],
        [ 2.3178, -0.4808, -1.8370],
        [-0.6030,  1.4281, -0.8252]], requires_grad=True)
tensor([5.8413e-07], requires_grad=True)


In [13]:
# high level way
# W, b initial
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
print(W)
print(b)
# optimizer
optimizer = optim.SGD([W, b], lr=0.1)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # cost function
    z = x_train.matmul(W) + b
    # F.cross_entropy contain softmax function. so don't need hypothesis
    cost = F.cross_entropy(z, y_train)
    # cost -> hypothesis update
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # print log
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
print(W)
print(b)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], requires_grad=True)
tensor([0.], requires_grad=True)
Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399963
tensor([[-3.6761, -0.1014,  3.7776],
        [-0.6601, -0.4641,  1.1242],
        [ 2.3178, -0.4808, -1.8370],
        [-0.6030,  1.4281, -0.8252]], requires_grad=True)
tensor([4.3958e-08], requires_grad=True)


In [16]:
# high level with nn
# set model
model = nn.Linear(4, 3)
print(list(model.parameters()))
# optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)
# training
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # hypothesis
    prediction = model(x_train)
    # cost function
    cost = F.cross_entropy(prediction, y_train)
    # cost -> hypothesis update
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # print log
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.4417,  0.4287, -0.0005,  0.1234],
        [ 0.0621,  0.0114,  0.4923,  0.4082],
        [-0.1306, -0.4836,  0.3241,  0.4842]], requires_grad=True), Parameter containing:
tensor([-9.5705e-02, -4.3368e-01,  2.7275e-04], requires_grad=True)]
Epoch    0/1000 Cost: 1.261718
Epoch  100/1000 Cost: 0.661285
Epoch  200/1000 Cost: 0.571155
Epoch  300/1000 Cost: 0.515474
Epoch  400/1000 Cost: 0.470755
Epoch  500/1000 Cost: 0.431270
Epoch  600/1000 Cost: 0.394584
Epoch  700/1000 Cost: 0.359179
Epoch  800/1000 Cost: 0.323865
Epoch  900/1000 Cost: 0.287807
Epoch 1000/1000 Cost: 0.252807
[Parameter containing:
tensor([[-2.8205, -0.3036,  2.6087, -0.3206],
        [ 0.6366, -0.1761, -0.1929,  1.3558],
        [ 2.5571,  0.4363, -1.5999, -0.0194]], requires_grad=True), Parameter containing:
tensor([-2.6071, -1.2388,  3.3168], requires_grad=True)]


In [18]:
# high level by class
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)
    def forward(self, x):
        return self.linear(x)
# get softmax classification model
model = SoftmaxClassifierModel()
print(list(model.parameters()))
# optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)
# training
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # hypothesis
    prediction = model(x_train)
    # cost function
    cost = F.cross_entropy(prediction, y_train)
    # cost -> hypothesis update
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # print log
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.1771,  0.0231,  0.2770, -0.0560],
        [-0.0496, -0.1705,  0.3163,  0.0286],
        [-0.0585,  0.4211,  0.3677, -0.2690]], requires_grad=True), Parameter containing:
tensor([-0.1974,  0.4381,  0.4043], requires_grad=True)]
Epoch    0/1000 Cost: 1.504225
Epoch  100/1000 Cost: 0.633273
Epoch  200/1000 Cost: 0.555836
Epoch  300/1000 Cost: 0.503012
Epoch  400/1000 Cost: 0.459287
Epoch  500/1000 Cost: 0.420105
Epoch  600/1000 Cost: 0.383284
Epoch  700/1000 Cost: 0.347372
Epoch  800/1000 Cost: 0.311258
Epoch  900/1000 Cost: 0.274674
Epoch 1000/1000 Cost: 0.245011
[Parameter containing:
tensor([[-3.0313, -0.2550,  2.6944, -0.7063],
        [ 0.2552, -0.1134, -0.1230,  0.9674],
        [ 2.4910,  0.6421, -1.6103, -0.5575]], requires_grad=True), Parameter containing:
tensor([-2.3998, -0.5911,  3.6358], requires_grad=True)]
